<a href="https://colab.research.google.com/github/gabiacuna/KL2021/blob/main/Analisis%20de%20Imagenes%20con%20DL/001-Autograd-Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fecha** : 2021-06-28

# Autograd: Diferenciación automática

El paquete `autograd` proporciona una diferenciación automática para todas las operaciones
en tensores. Es un marco de definición por ejecución, lo que significa que su backprop es
definido por cómo se ejecuta su código, y que cada iteración puede ser
diferente.

In [1]:
import torch

Creando a un tensor:

In [2]:
# Cree un tensor 2x2 con capacidades de acumulación de gradientes
x = torch.tensor([[1, 2], [3, 4]], requires_grad=True, dtype=torch.float32)
print(x)

tensor([[1., 2.],
        [3., 4.]], requires_grad=True)


Ahora podemos hacer operaciónes sobre los tensores

In [3]:
# Deduct 2 from all elements
y = x - 2
print(y)

tensor([[-1.,  0.],
        [ 1.,  2.]], grad_fn=<SubBackward0>)


`y` fue creada como resultado de una operación, y por ello tiene ele atributo `grad_fn`:

In [4]:
print(y.grad_fn)

In [5]:
# ¿Qué pasa acá?
print(x.grad_fn)    #Sale none pok x no depende de y, solo tiene requires_grad=True

None


In [6]:
# Let's dig further...
y.grad_fn

In [7]:
y.grad_fn.next_functions[0][0]

In [8]:
y.grad_fn.next_functions[0][0].variable

tensor([[1., 2.],
        [3., 4.]], requires_grad=True)

In [9]:
# Podemos aplicar más funciones sobre `y`
z = y * y * 3
a = z.mean()  # promedio

print(z)
print(a)

tensor([[ 3.,  0.],
        [ 3., 12.]], grad_fn=<MulBackward0>)
tensor(4.5000, grad_fn=<MeanBackward0>)


## Gradients

Ahora podemos retroceder con `out.backward()`, lo cual es equivalente a
`out.backward(torch.tensor([1.0]))`.

In [10]:
# Backprop
a.backward()

Print gradients $\frac{\text{d}a}{\text{d}x}$.




In [11]:
print(x.grad)

tensor([[-1.5000,  0.0000],
        [ 1.5000,  3.0000]])


In [12]:
x = torch.randn(3, requires_grad=True)

y = x * 2
i = 0
while y.data.norm() < 1000:
    y = y * 2
    i += 1
print(y)

tensor([1829.9207,  353.0065,  637.1857], grad_fn=<MulBackward0>)


In [13]:
# Si no corremos hacia atrás en un escalar, necesitamos especificar el grad_output
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


ojito que los `.backward()` se pueden correr solo una vez.

In [17]:
print(i)

9


## Inferencia

In [15]:
# Esta variable decide el rango del tensor por debajo de
n = 3

In [19]:
x, w, z

(tensor([1., 2., 3.], requires_grad=True),
 tensor([1., 1., 1.], requires_grad=True),
 tensor(6., grad_fn=<DotBackward>))

In [16]:
# Tanto `x` como `w` que permiten la acumulación de gradientes.
x = torch.arange(1., n + 1, requires_grad=True)
w = torch.ones(n, requires_grad=True)
z = w @ x   #Producto punto 
z.backward()
print(x.grad, w.grad, sep='\n')

tensor([1., 1., 1.])
tensor([1., 2., 3.])


In [21]:
# Solo `w` que permite la acumulación de gradiente
x = torch.arange(1., n + 1)
w = torch.ones(n, requires_grad=True)
z = w @ x
z.backward()
print(x.grad, w.grad, sep='\n')

None
tensor([1., 2., 3.])


In [22]:
x = torch.arange(1., n + 1)
w = torch.ones(n, requires_grad=True)

# Independientemente de lo que haga en este contexto, todos los tensores de la
# antorcha no tendrán acumulación de gradiente
with torch.no_grad():
    z = w @ x

try:
    z.backward()  # PyTorch arrojará un error aquí, ya que z no tiene acumulaciones graduales.
except RuntimeError as e:
    print('RuntimeError!!! >:[')
    print(e)

RuntimeError!!! >:[
element 0 of tensors does not require grad and does not have a grad_fn


In [23]:
z

tensor(6.)

## More stuff

La documentación del paquete de diferenciación automática (`autograd`) se encuentra en
http://pytorch.org/docs/autograd.